# Very Effective when used with 5EMA

## Backtest for the buying stratergy 
<hr>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import talib as ta
import yfinance as yf
import datetime

In [40]:
data = yf.download(tickers='ICICIBANK.NS', period='60d', interval='15m')
data.to_csv("test1.csv")

[*********************100%***********************]  1 of 1 completed


In [41]:
data = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/test1.csv')
# data = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/Data/NIFTY 50_5min.csv')
# data = data[data['Datetime'].dt.year==2021]
data['Datetime'] = pd.to_datetime(data['Datetime'])
data['5EMA'] = ta.EMA(data['Close'],5)

In [42]:
data = data[data['Datetime'].dt.month==1]
# data = data.iloc[25:]
data

,Datetime,Open,High,Low,Close,Adj Close,Volume,5EMA
1354,2023-01-02 09:15:00,896.799988,897.900024,894.099976,895.650024,895.650024,210251,893.667937
1355,2023-01-02 09:30:00,895.700012,903.349976,895.299988,903.349976,903.349976,453679,896.895283
1356,2023-01-02 09:45:00,903.400024,903.950012,902.000000,902.900024,902.900024,172059,898.896863
1357,2023-01-02 10:00:00,902.650024,905.000000,902.549988,903.299988,903.299988,163028,900.364572
1358,2023-01-02 10:15:00,903.250000,904.000000,902.599976,902.700012,902.700012,154544,901.143052
...,...,...,...,...,...,...,...,...
1475,2023-01-06 14:30:00,865.700012,866.000000,865.000000,865.450012,865.450012,451270,864.670670
1476,2023-01-06 14:45:00,865.450012,868.349976,865.299988,867.650024,867.650024,683490,865.663788
1477,2023-01-06 15:00:00,868.049988,872.000000,867.750000,871.250000,871.250000,914212,867.525859
1478,2023-01-06 15:15:00,871.349976,871.950012,868.799988,870.500000,870.500000,854708,868.517239


In [43]:
def enter(data,j,sl,entry,target,arr,arrc,ana,count,sl_m):
    k = j+2
    while(k<len(data['Close'])):
        
        if data['Datetime'].dt.date.values[k] not in sl_m:
            sl_m[data['Datetime'].dt.date.values[k]] = 0
            
        if(data['Datetime'].dt.day.values[k]>data['Datetime'].dt.day.values[j]):
            return 

        if(data['Close'].values[k]<sl):
            change = (sl-entry)/entry
            arrc.append(data['Datetime'].values[k])
            arr.append(change)
            sl_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "-ve sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{6}, entry-time{4} , exit-time:{5}".format(sl,entry,sl,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],data['Datetime'].dt.date.values[k])
            return k 

        if(data['Close'].values[k]>target):
            change = (target-entry)/entry
            arrc.append(data['Datetime'].values[k])
            arr.append(change)
            ana[count] = "+ve sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{6}, entry-time{4} , exit-time:{5}".format(sl,entry,target,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],data['Datetime'].dt.date.values[k])
            return k

        if(data['5EMA'].values[k]-data['High'].values[k]>0.4):
            change = (data['Close'].values[k]-entry)/entry
            arrc.append(data['Datetime'].values[k])
            arr.append(change)
            ch_b = change>0
            if(change<0):
                sl_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "{6} sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{7}, entry-time{4} , exit-time:{5}".format(sl,entry,data['Close'].values[k],target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],ch_b,data['Datetime'].dt.date.values[k])
            return k-2

        k += 1

In [44]:
def trade(data,i,arr,arrc,ana,count,sl_m):
    sl = 0.0
    sl_h = 0.0
    entry = 0.0
    target = 0.0
    j = i+1
    while(j<len(data['Close'])-1):
        if(data['High'].values[j]>data['5EMA'].values[j] and data['Low'].values[j]<data['5EMA'].values[j]):
            sl = data['Low'].values[j-1]
            entry = data['Open'].values[j+1] + ((data['Close'].values[j+1]-data['Open'].values[j+1])/2)
            target = entry + (3*(entry-sl))
            if(entry<sl):
                return j 
            return enter(data,j,sl,entry,target,arr,arrc,ana,count,sl_m)
        j += 1

In [45]:
def EMA(amount,arr,data,arrc,ana,sl_m):
    i = 0
    count = 0
    while(i<len(data['Close'])-1):
        
        if data['Datetime'].dt.date.values[i] not in sl_m:
            sl_m[data['Datetime'].dt.date.values[i]] = 0
            
        if(sl_m[data['Datetime'].dt.date.values[i]]>3):
            i += 1
            continue
            
        if(data['High'].values[i]<data['5EMA'].values[i]):
            count += 1
            a = trade(data,i,arr,arrc,ana,count,sl_m)
            try:
                b = a-i+1
                i += b
                continue
            except:
                c = 1+1
        i += 1

In [46]:
amount  = 4000
arr = []
arrc = []
ana = {}
sl_m = {}
EMA(amount,arr,data,arrc,ana,sl_m)  

In [47]:
amount  = 10000
ini = amount
for i in arr:
    amount = (1+i)*amount
print(ini)
print(amount)
print(amount-ini)
print(((amount-ini)/ini)*100)

10000
10067.574658003812
67.57465800381215
0.6757465800381214


In [48]:
arr

[0.004259326539294553,
 0.004017001307808625,
 -0.0021278060228325274,
 0.0006057568980852836]

In [49]:
sl_m

{datetime.date(2023, 1, 2): 0,
 datetime.date(2023, 1, 3): 0,
 datetime.date(2023, 1, 4): 0,
 datetime.date(2023, 1, 5): 1,
 datetime.date(2023, 1, 6): 0}

In [50]:
ana

{2: '+ve sl:896.75 , entry:898.0249938964844 , exit:901.8499755859375 , target:901.8499755859375 , Date:2023-01-03, entry-time13:00:00 , exit-time:15:00:00',
 3: '+ve sl:895.0 , entry:896.2000122070312 , exit:899.800048828125 , target:899.800048828125 , Date:2023-01-04, entry-time13:00:00 , exit-time:14:00:00',
 4: '-ve sl:891.0499877929688 , entry:892.9500122070312 , exit:891.0499877929688 , target:898.6500854492188 , Date:2023-01-05, entry-time10:30:00 , exit-time:10:45:00',
 8: '+ve sl:866.5999755859375 , entry:866.7749938964844 , exit:867.300048828125 , target:867.300048828125 , Date:2023-01-06, entry-time11:30:00 , exit-time:12:00:00'}